In [3]:
import pandas as pd
import numpy as np
from scipy import stats
import ast
import networkx as nx
from pyvis.network import Network
import graphviz
import seaborn as sns
from statannotations.Annotator import Annotator
import matplotlib.pyplot as plt
from IPython.display import HTML
plt.style.use('tableau-colorblind10')
import numba
import seaborn as sns
import iqplot
from sklearn import metrics
import bokeh.models
import bokeh.plotting
import bokeh.io
bokeh.io.output_notebook()

Loading BokehJS ...

In [4]:
gnnbase = pd.read_csv('baseGNNRuntimes.csv')
gnnhetero = pd.read_csv('heteroGNNRuntimes.csv')
gnnunfair = pd.read_csv('unfairGNNRuntimes.csv')
gnnless = pd.read_csv('lessinformGNNRuntimes.csv')
gnnmore = pd.read_csv('moreinformGNNRuntimes.csv')
gnns25 = pd.read_csv('SERGIO25GNNRuntimes.csv')
gnns50 = pd.read_csv('SERGIO50GNNRuntimes.csv')
gnnwis = pd.read_csv('WisconsinGNNRuntimes.csv')
gnntex = pd.read_csv('TexasGNNRuntimes.csv')
gnncora = pd.read_csv('CoraGNNRuntimes.csv')
gnncs = pd.read_csv('CiteSeerGNNRuntimes.csv')
gnnq = pd.read_csv('QuestionsGNNRuntimes.csv')

In [6]:
betabase = pd.read_csv('baseBetaRuntimes.csv')
betahetero = pd.read_csv('heteroBetaRuntimes.csv')
betaunfair = pd.read_csv('unfairBetaRuntimes.csv')
betaless = pd.read_csv('lessinformBetaRuntimes.csv')
betamore = pd.read_csv('moreinformBetaRuntimes.csv')
betas25 = pd.read_csv('SERGIO25BetaRuntimes.csv')
betasb25 = pd.read_csv('SERGIO25BetaBatchRuntimes.csv')
betas50 = pd.read_csv('SERGIO50BetaRuntimes.csv')
betasb50 = pd.read_csv('SERGIO25BetaBatchRuntimes.csv')
betasb25['Data'] = '25% Sparsity Batches'
betasb50['Data'] = '50% Sparsity Batches'
betawis = pd.read_csv('WisconsinBetaRuntimes.csv')
betatex = pd.read_csv('TexasBetaRuntimes.csv')
betacora = pd.read_csv('CoraBetaRuntimes.csv')
betacs = pd.read_csv('CiteSeerBetaRuntimes.csv')
#betaq = pd.read_csv('QuestionsBetaRuntimes.csv')

In [7]:
frames = [gnnbase,gnnhetero,gnnunfair,gnnless,gnnmore,gnns25,gnns50,
          gnnwis,gnntex,gnnwis,gnntex,gnncora,gnncs,betabase,betahetero,
          betaunfair,betaless,betamore,betas25,betasb25,betas50,betasb50,
          betawis,betatex,betacora,betacs,gnnq]
# frames = [gnnbase,gnnhetero,gnnunfair,gnnless,gnnmore,gnns25,gnns50,
#           gnnwis,gnntex,gnnwis,gnntex,gnncora,gnncs,betabase,betahetero,
#           betaunfair,betaless,betamore,betas25,betasb25,betas50,betasb50,
#           betawis,betatex,betacora,betacs,gnnq,betaq]
df = pd.concat(frames)
df = df.rename({'Data':'Dataset'},axis=1)
#df['Dataset'] = df['Dataset'].apply({'Questions':'Questions','25% Sparsity Batches':'25% Sparsity Batches', '50% Sparsity Batches':'50% Sparsity Batches',25:'25% Sparsity', 50:'50% Sparsity', 'CiteSeer': 'CiteSeer', 'Cora': 'Cora', 'Texas': 'Texas', 'Wisconsin': 'Wisconsin', 'base': 'SG-BASE', 'hetero': 'SG-HETEROPHILIC','lessinform': 'SG-LESSINFORM', 'moreinform': 'SG-MOREINFORM','unfair': 'SG-UNFAIR'})
df.head()

,Unnamed: 0,Dataset,Explainer,Experiment,# Epochs,Full Runtime,Average Runtime
0,0,base,GNN,0,50,3.233512,0.064670
1,1,base,GNN,1,50,2.405525,0.048110
2,2,base,GNN,2,50,1.916023,0.038320
3,3,base,GNN,3,50,1.927296,0.038546
4,4,base,GNN,4,50,1.790859,0.035817


In [8]:
df

,Unnamed: 0,Dataset,Explainer,Experiment,# Epochs,Full Runtime,Average Runtime
0,0,base,GNN,0,50,3.233512,0.064670
1,1,base,GNN,1,50,2.405525,0.048110
2,2,base,GNN,2,50,1.916023,0.038320
3,3,base,GNN,3,50,1.927296,0.038546
4,4,base,GNN,4,50,1.790859,0.035817
...,...,...,...,...,...,...,...
20,20,Questions,GNN,20,50,123.224573,2.464491
21,21,Questions,GNN,21,50,121.247921,2.424958
22,22,Questions,GNN,22,50,127.941621,2.558832
23,23,Questions,GNN,23,50,113.817421,2.276348


In [9]:
d = list(df['Dataset'])
dataset=[]
for el in d:
    if el not in dataset:
        dataset.append(el)
print(dataset)

['base', 'hetero', 'unfair', 'lessinform', 'moreinform', 25, 50, 'Wisconsin', 'Texas', 'Cora', 'CiteSeer', '25% Sparsity Batches', '50% Sparsity Batches', 'Questions']


In [10]:
for data in dataset:
    curr = df[df['Dataset'] == data]
    if data != '25% Sparsity Batches' and data != '50% Sparsity Batches':
        exp = ['GNN','Beta']
    else:
        exp = ['Beta']
    for ex in exp:
        c = curr[curr['Explainer'] == ex]
        full = np.mean(list(c['Full Runtime']))
        avg = np.mean(list(c['Average Runtime']))
        print(f'Data: {data}, Exp: {ex}, Full: {full}, Avg: {avg}')

Data: base, Exp: GNN, Full: 2.092942976951599, Avg: 0.041858859539031935
Data: base, Exp: Beta, Full: 11.18047643184662, Avg: 0.2236095286369323
Data: hetero, Exp: GNN, Full: 3.3474967765808104, Avg: 0.06694993553161618
Data: hetero, Exp: Beta, Full: 15.41835412979126, Avg: 0.3083670825958251
Data: unfair, Exp: GNN, Full: 2.8587034130096436, Avg: 0.05717406826019282
Data: unfair, Exp: Beta, Full: 21.758967599868775, Avg: 0.43517935199737545
Data: lessinform, Exp: GNN, Full: 2.9631510543823243, Avg: 0.05926302108764643
Data: lessinform, Exp: Beta, Full: 11.441699342727661, Avg: 0.2288339868545532
Data: moreinform, Exp: GNN, Full: 2.0722943782806396, Avg: 0.041445887565612745
Data: moreinform, Exp: Beta, Full: 14.12384494304657, Avg: 0.28247689886093136
Data: 25, Exp: GNN, Full: 0.12746546745300288, Avg: 0.00509861869812006
Data: 25, Exp: Beta, Full: 514.8820004272461, Avg: 10.297640008544922
Data: 50, Exp: GNN, Full: 0.03186197280883785, Avg: 0.00127447891235346
Data: 50, Exp: Beta, Ful

C:\Users\whitn\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\whitn\anaconda3\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
